# Aproximação de Vogel 

**Professor:** Luciano Guimarães de Castro 

### Lucas Machado Moschen

In [2]:
using Pkg
pkg"activate ../."
pkg"instantiate"
using Plots, LinearAlgebra

  Resolving package versions...
No Changes to `~/Documents/GitHub/linear-programming/Project.toml`
No Changes to `~/Documents/GitHub/linear-programming/Manifest.toml`
 Activating environment at `~/Documents/GitHub/linear-programming/Project.toml`


## Entrada

`costs`: array 2d (m x n). 

Ex.: 
$c = \begin{bmatrix} 
6 & 6 & 9 & 4 & 10 \\
3 & 2 & 7 & 5 & 12 \\
8 & 7 & 5 & 6 & 4   \\
11 & 12 & 9 & 5 & 2 \\
4 & 3 & 4 & 5 & 11  \\
\end{bmatrix}
$

`demand`: array 1d. 

Ex.: $d = [120, 80, 50, 75, 85]$ 


`supply`: array 1d.

Ex.: $s = [45, 90, 95, 75, 105]$


## Constrói arrays com penalidades

- Para cada linha, encontre os dois menore e tome a diferença absoluta entre eles. Armazene esse valor na linha correspondente.

- Para cada coluna, encontre os dois menore e tome a diferença absoluta entre eles. Armazena esse valor na coluna correspondente. 

In [183]:
function build_penalty!(m::Int64, n::Int64, 
                        row_diff::Array{Float64,1}, 
                        col_diff::Array{Float64,1},
                        cost::Array{Float64,2})
    """
    m/n: Int64
        Number of rows/columns.
    row/col_diff: Array{Float64,1}
        Row/column difference array.
    cost: Array{Float64,2}
        Each row represents the supply equation and each column represents the demand equation.
    """
    for row = 1:m
        row_diff[row] = partialsort(cost[row,:], 2) - partialsort(cost[row,:], 1)
    end
    for col = 1:n
        col_diff[col] = partialsort(cost[:,col], 2) - partialsort(cost[:,col], 1)
    end
end;

## Seleciona célula

Após construir as arrays de penalidade, tomamos a penalidade máxima e a sua linha/coluna referente. Tomamos a célula com o menor custo nessa linha/coluna.

In [184]:
function select_cell(row_diff::Array{Float64,1}, col_diff::Array{Float64,1}, cost::Array{Float64,2})
    """
    row/col_diff: Array{Float64,1}
        Row/Column difference array.
    cost: Array{Float64,2}
        Each row represents the supply equation and each column represents the demand equation.
    """
    max_penalty_row = findmax(row_diff)
    max_penalty_col = findmax(col_diff)
    if max_penalty_row[1] < max_penalty_col[1] 
        col = max_penalty_col[2]
        row = findmin(cost[:,col])[2]
    else 
        row = max_penalty_row[2]
        col = findmin(cost[row,:])[2]
    end
    return (row, col)
end;

## Define x0 

Definimos x0 como o menor entre o suprimento e a demanda e inutilizamos a linha/coluna que teve demanda suprida (caso demanda seja menor) ou suprimento dado (caso suprimento seja menor).

In [198]:
function define_x0!(x0::Array{Float64,2}, 
                    supply::Array{Float64,1}, 
                    demand::Array{Float64,1}, 
                    row::Int64, col::Int64, 
                    cost::Array{Float64,2}, 
                    M::Float64)
    """
    x0: Array{Float64,2}
        Current initial value.
    supply/demand: Array{Float64,1}
        Supply and demand array.
    row/col: Int64
        Row and column selected.
    cost: Array{Float64,2}) 
        Each row represents the supply equation and each column represents the demand equation.
    M: Float64 
        Maximum value of cost array
    """
    x0[row, col] = min(supply[row], demand[col])
    supply[row] -= x0[row, col]
    demand[col] -= x0[row, col]

    if supply[row] > demand[col]
        cost[:,col] .= M
    elseif supply[row] < demand[col]
        cost[row,:] .= M
    else
        cost[row,:] .= M
        cost[:,col] .= M
    end
end;

## Juntando tudo 

In [199]:
function vogel_approximation!(cost::Array{Float64,2}, 
                              demand::Array{Float64,1}, 
                              supply::Array{Float64,1})::Array{Float64,2}
    """
    cost_matrix: Array{Float64,2}. 
        Each row represents the supply equation and each column represents the demand equation.
    demand: Array{Float64,1}. 
        Values in the right side of demand equation. 
    supply: Array{Float64,1}.
        Values in the right side of supply equation. 
    """
    m = size(cost)[1]
    n = size(cost)[2]
    
    x0 = zeros(m,n)
    
    row_diff = zeros(m)
    col_diff = zeros(n)
    
    M = maximum(cost)
    
    while (sum(supply) != 0) || (sum(demand) != 0)
        
        build_penalty!(m, n, row_diff, col_diff, cost)
        row, col = select_cell(row_diff, col_diff, cost)
        
        define_x0!(x0, supply, demand, row, col, cost, M)
    end
    return x0
end;

vogel_approximation! (generic function with 2 methods)

## Exemplos 

In [200]:
c = [3. 1 7 4;
     2 6 5 9;
     8 3 3 2];
    
d = [250.,350, 400, 200];

s = [300.,400, 500];

In [201]:
vogel_approximation!(c, d, s)

3×4 Array{Float64,2}:
   0.0  300.0    0.0    0.0
 250.0    0.0  150.0    0.0
   0.0   50.0  250.0  200.0

In [205]:
costs = [6. 6 9 4 10;
         3 2 7 5 12;
         8 7 5 6 4;
         11 12 9 5 2;
         4 3 4 5 11];
demand = [120., 80, 50, 75, 85];
supply = [45., 90, 95, 75, 105];

In [206]:
vogel_approximation!(costs, demand, supply)

5×5 Array{Float64,2}:
   0.0   0.0   0.0  45.0   0.0
  10.0  80.0   0.0   0.0   0.0
   5.0   0.0  50.0  30.0  10.0
   0.0   0.0   0.0   0.0  75.0
 105.0   0.0   0.0   0.0   0.0